<a href="https://colab.research.google.com/github/fbeilstein/dbms/blob/master/db_lecture_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Failure Detection**

Failures should be detected in a timely manner. A faulty process might get contacted even though it won’t be able to respond, increasing latencies and reducing overall system availability.

Detecting failures in asynchronous distributed systems (i.e., without making any timing assumptions) is extremely difficult as it’s impossible to tell whether the process has crashed, or is running slowly and taking an indefinitely long time to respond (FLP impossibility, previous lecture). 

* process that has stopped executing its steps completely: dead, failed or crashed
* suspected process (may be dead): unresponsive, faulty, slow

Failures may occur 
* link level (messages between processes are lost or delivered slowly)
* process level (the process crashes or is running slowly)


Slowness may not always be distinguishable from failure = trade-off between wrongly suspecting alive processes as dead and delaying marking an unresponsive process as dead.

A **failure detector** is a local subsystem responsible for identifying failed or unreachable processes to exclude them from the algorithm and guarantee liveness while preserving safety.

**Liveness** and **safety** are the properties that describe an algorithm’s ability to solve a specific problem and the correctness of its output. 

* **liveness** guarantees that a specific intended event must occur (e.g. process failed -> failure detected).
* **safety** guarantees that unintended events will not occur (e.g. considered dead -> really dead). 

We want from failure detector **efficiency** (detect fails fast) and **accuracy** (detect fails precise). We can think of the relationship between efficiency and accuracy as a tunable parameter: a more efficient algorithm might be less precise, and a more accurate algorithm is usually less efficient. 


Often failure detectors are allowed to produce false-positives and assume the **absence of Byzantine failures** (processes do not attempt to intentionally lie about their state)

Failure detectors are an essential prerequisite and an integral part of many consensus and atomic broadcast algorithms.

**Heartbeats and Pings**

We can query the state of remote processes by
* **ping**, which sends messages to remote processes, checking if they are still alive by expecting a response within a specified time period.
* **heartbeat** when the process is actively notifying its peers that it’s
still running by sending messages to them.

Each process maintains a list of other processes (alive, dead, and suspected ones) and updates it with the last response time for each process. If a process fails to respond to a ping message or produce heartbeat for a longer time, it is marked as suspected.

Many failure-detection algorithms are based on heartbeats and timeouts. For example, [Akka](https://akka.io/), a popular framework for building distributed systems, has an implementation of a deadline failure detector, which uses heartbeats.

This approach has several potential downsides: 
* precision relies on the careful selection of ping frequency and timeout
* does not capture process visibility from the perspective of other processes

**Timeout-Free Failure Detector**

Some algorithms avoid relying on timeouts for detecting failures. For example [Aguilera, Marcos K., Wei Chen, and Sam Toueg. **1997**. “Heartbeat: a Timeout-Free Failure Detector for Quiescent Reliable Communication.”](https://www.microsoft.com/en-us/research/uploads/prod/1997/09/wdag97_hb.pdf)

Assumptions:
* any two correct processes are connected to each other with a fair path, which contains only fair links (i.e., if a message is sent over this link infinitely often, it is also received infinitely often)
* each process is aware of the existence of all other processes in the network.


Each process maintains a list of neighbors and counters associated with them. 
1. Send initial message (contains the first sender in the path and a unique identifier that can be used to avoid broadcasting the same message multiple times)
2. If new message received:
  - append itself to the path
  - send the heartbeat to the ones that are not present there (note: propagation stops if all known processes received it)
  - increments counters for all participants present in the path (note: OK processes have unbounded heartbeats, fault -- bounded)
 
Since messages are propagated through different processes, and heartbeat paths contain aggregated information received from the neighbors, we can (correctly) mark an unreachable process as alive even when the direct link between the two processes is faulty.

Heartbeat counters represent a global and normalized view of the system. This view captures how the heartbeats are propagated relative to one another, allowing us to compare processes. However, one of the shortcomings of this approach is that interpreting heartbeat counters may be quite tricky: we need to pick a threshold that can yield reliable results. Unless we can do that, the algorithm will falsely mark active processes as suspected.

**Outsourced Heartbeats**


An alternative approach, used by the [Scalable Weakly Consistent Infection-style Process Group Membership Protocol (SWIM)](https://dl.acm.org/doi/pdf/10.1145/3361525.3361556) is to use outsourced heartbeats to **improve reliability** using information about the process liveness from the
perspective of its neighbors. 


This approach does not require processes to be aware of all other processes in the network, only a subset of connected peers.

Suppose P1,P2,P3 neighbours of PB

P1 -ping-> PB. If alive mark as ok. If not: P1 -message-> P2,P3 -ping-> PB. If alive P2,P3 -message-> P1.  

This allows accounting for both direct and indirect reachability, we can check the state of PB from the perspective of P1,P2,P3.

Outsourced heartbeats allow reliable failure detection by distributing responsibility for deciding across the group of members. 
* does not require broadcasting messages to a broad group of peers (choose few randomly). 
* can be triggered in parallel
* allow us to make more accurate decisions.

**Phi-Accrual Failure Detector**

Used in many distributed systems, e.g. **Cassandra** and **Akka** (along with the aforementioned deadline failure detector).

Instead of treating node failure as a binary problem, where the process can be only in two states: up or down, a [phi-accrual (φ-accrual) failure detector](https://oneofus.la/have-emacs-will-hack/files/HDY04.pdf) has a continuous scale, capturing the probability of the monitored process’s crash. 

It works by maintaining a sliding window, collecting arrival times of the most recent heartbeats from the peer processes. This information is used to approximate arrival time of the next heartbeat, compare this approximation with the actual arrival time, and compute the suspicion level φ: how certain the failure detector is about the failure, given the current network conditions. If this value reaches a threshold, the node is marked as down.

This failure detector dynamically adapts to changing network conditions by adjusting the scale on which the node can be marked as a suspect. 

From the architecture perspective, a phi-accrual failure detector can be viewed as a combination of three subsystems:
* **Monitoring**
Collecting liveness information through pings, heartbeats, or request-response
sampling.
* **Interpretation**
Making a decision on whether or not the process should be marked as suspected.
* **Action**
A callback executed whenever the process is marked as suspected. 


The monitoring process collects and stores data samples (which are assumed to follow a normal distribution) in a fixed-size window of heartbeat arrival times. Newer arrivals are added to the window, and the oldest heartbeat data points are discarded. Distribution parameters are estimated from the sampling window by determining the **mean** and **variance** of samples. This information is used to compute the probability of arrival of the message within t time units after the previous one. Given this information, we compute φ, which describes how likely we are to make a correct decision about a process’s liveness. In other words, how likely it is to make a mistake and receive a heartbeat that will contradict the calculated assumptions.



**Gossip and Failure Detection**

Another approach that avoids relying on a single-node view to make a decision is a gossip-style failure detection service (van Renesse, Robbert, Yaron Minsky, and Mark Hayden. 1998. “A Gossip-Style Failure Detection Service.”), which uses gossip (analogy: think about virus spreading) to collect and distribute states of neighboring processes.

* Each process maintains table

Neighbour | Last Heartbeat timestamp
---|---
P1 (self) | t1
P2 | t2
P3 | t3

* Periodically, each member increments its heartbeat counter and distributes its list to a random neighbor. 

* Upon the message receipt, the neighboring node merges the list with its own, updating heartbeat counters for the other neighbors.

* If any node did not update its counter for long enough, it is considered failed. 

Failure timeout and communication frequency should be carefully chosen.

Note: bandwidth is capped, and can grow at most linearly with a number of processes in the system.


Case study: consider 3 processes P1,P2,P3.
* All three can communicate and update their timestamps.
* Link P1-P3 is lost. Its state is still propagated through P2.
* P3 crashes. Since it doesn’t send updates anymore, it is detected as failed by
other processes.

This way, we can detect crashed nodes, as well as the nodes that are unreachable by any other cluster member. This decision is reliable, since the view of the cluster is an aggregate from multiple nodes. If there’s a link failure between the two hosts, heartbeats can still propagate through other processes. Using gossip for propagating system states increases the number of messages in the system, but allows information to spread more reliably

**Reversing Failure Detection Problem** 

Since propagating the information about failures is not always possible, and propagating it by notifying every member might be expensive, one of the approaches, called [FUSE (failure notification service)](https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.59.2061), focuses on reliable and cheap
failure propagation that works even in cases of network partitions.


This approach arranges all active processes in groups. Every time a single process failure is detected, it is converted and propagated as a group failure. This allows detecting failures in the presence of any pattern of disconnects, partitions, and node failures.

* send pings to members of group
* if anyone did not respond - stop responding as well


All failures are propagated through the system from the source of failure to all other participants. Participants gradually stop responding to pings, converting from the individual node failure to the group failure.

(+) every member is guaranteed to learn about group failure and adequately react to it. 


(-) link failure separating a single process from other ones can be converted to the group failure as well

**Leader Election**

* Synchronization can be quite costly: if each algorithm step involves contacting each other participant, we can end up with a significant communication overhead. This is particularly true in large and geographically distributed networks. 
* Some algorithms rely on the existence of the leader (sometimes called coordinator) process, responsible for executing or coordinating steps of a distributed algorithm.


Usually, the process remains a leader until it crashes. After the crash, any other process can start a new election round, assume leadership, if it gets elected, and continue the failed leader’s work.


We expect
* **liveness** of the election algorithm (guarantees that most of the time there will be a leader, and the election will eventually complete)
* **safety** - guarantee there may be at most one leader at a time, and completely eliminate the possibility of a **split brain** situation (when two
leaders serving the same purpose are elected but unaware of each other). 

Leader processes can: order messages and disseminate them among the processes, coordinate system reorganization after the failure, during initialization, or when important state changes happen.


Having a stable leader in the system helps to avoid state synchronization between remote participants, reduce the number of exchanged messages, and drive execution from a single process instead of requiring peer-to-peer coordination. 


One of the potential **problems** in systems with a notion of leadership is that the leader can become a bottleneck. To overcome that, many systems partition data in nonintersecting independent replica sets (partitioning).
Instead of having a single system-wide leader, each replica set has its own leader. One of the systems that uses this approach is [Spanner](https://en.wikipedia.org/wiki/Spanner_(database)).

**Bully Algorithm**

[See "Elections in a Distributed Computing System" for few variants](http://vis.usal.es/rodrigo/documentos/papers/BullyAlgorithm.pdf)

* Each process gets a unique rank assigned to it. 
* During the election, the process with the highest rank becomes a leader.

This algorithm is known for its simplicity. The algorithm is named bully because the highest-ranked node “bullies” other nodes into accepting it. It is also known as monarchial leader election: the highest-ranked sibling becomes a monarch after the previous one ceases to exist.



1. The process sends election messages to processes with higher identifiers.
2. The process waits, allowing higher-ranked processes to respond. If no higher-ranked process responds, it proceeds with step 3. Otherwise, the process notifies the highest-ranked process it has heard from, and allows it to proceed with step 3.
3. The process assumes that there are no active processes with a higher rank, and notifies all lower-ranked processes about the new leader.


Consider processes with ranks 1,2,3,4,5,6. Initial leader was 6. 6 crashes. Look, for example, process 3. 3 -ping-> 4,5,6. 4,5-alive-> 3. 3 -notify-> 5. 5 -notify-> 1,2,3,4.

**downsides**
* violates the safety guarantee in the presence of network partitions (2 components elect 2 leaders). This situation is called **split brain**.
* An unstable high-ranked node proposes itself as a leader, fails shortly
thereafter, wins reelection, fails again, and the whole process repeats. This problem can be solved by distributing host quality metrics and taking them into consideration during the election.

**Next-In-Line Failover**

There are many versions of the bully algorithm that improve its various properties. For example, we can use multiple next-in-line alternative processes as a failover to shorten reelections.

* Each elected leader provides a list of failover nodes. 
* When one of the processes detects a leader failure, it starts a new election round by sending a message to the highest-ranked alternative from the list provided by the failed leader. 
* If one of the proposed alternatives is up, it becomes a new leader without having to go through the complete election round. 
* If the process that has detected the leader failure is itself the highest ranked process from the list, it can notify the processes about the new leader right away.

Consider processes with ranks 1,2,3,4,5,6. Initial leader was 6, it provides alternative 5. 6 crashes. Look, for example, process 3. 3 -ping-> 5. 5-alive-> 3. 3 -notify-> 5. 5 -notify-> 1,2,3,4.

As a result, we require fewer steps during the election if the next-in-line process is alive

**Candidate/Ordinary Optimization**


Another algorithm attempts to lower requirements on the number of messages by
splitting the nodes into two subsets, candidate and ordinary, where only one of the candidate nodes can eventually become a leader.


The ordinary process initiates election by contacting candidate nodes, collecting responses from them, picking the highest-ranked alive candidate as a new leader, and then notifying the rest of the nodes about the election results.


To solve the problem with multiple simultaneous elections, the algorithm proposes to use a tiebreaker variable δ, a process-specific delay, varying significantly between the nodes, that allows one of the nodes to initiate the election before the other ones. The tiebreaker time is generally greater than the message round-trip time. Nodes with higher priorities have a lower δ, and vice versa.


Consider processes with ranks 1,2,3,4,5,6. Initial leader was 6, candidates = 1,2,6, ordinary=3,4,5. 6 crashes. Look, for example, process 3. 3 -ping-> 1,2. 1,2-alive-> 3. 2 new leader. 3 -notify-> 1,2,4,5.


**Invitation Algorithm**


An invitation algorithm allows processes to “invite” other processes to join their groups instead of trying to outrank them. 

This algorithm allows multiple leaders by definition, since each group has its own leader.

Each process starts as a leader of a new group, where the only member is the process itself. Group leaders contact peers that do not belong to their groups, inviting them to join. If the peer process is a leader itself, two groups are merged. Otherwise, the contacted process responds with a group leader ID, allowing two group leaders to establish contact and merge groups in fewer steps.

If peer gets 2 or more invitation it may accept any (groups will grow and merge into one at the end).

Since groups are merged, it doesn’t matter whether the process that suggested the group merge becomes a new leader or the other one does. To keep the number of messages required to merge groups to a minimum, a leader of a larger group can become a leader for a new group. This way only the processes from the smaller group have to be notified about the change of leader.


Similar to the other discussed algorithms, this algorithm allows processes to settle in multiple groups and have multiple leaders. The invitation algorithm allows creating process groups and merging them without having to trigger a new election from scratch, reducing the number of messages required to finish the election.

**Ring Algorithm**


In the [ring algorithm](https://dl.acm.org/doi/pdf/10.1145/359104.359108), all nodes in the system form a ring and are aware of the ring topology (i.e., their predecessors and successors in the ring). 

When the process detects the leader failure, it starts the new election. The election message is forwarded across the ring: each process contacts its successor (the next node closest to it in the ring). If this node is unavailable, the process skips the unreachable node and attempts to contact the nodes after it in the ring, until eventually one of them responds. Nodes contact their siblings, following around the ring and collecting the live node
set, adding themselves to the set before passing it over to the next node, similar to the failure-detection algorithm described in “Timeout-Free Failure Detector”, where nodes append their identifiers to the path before passing it to the next node.

The algorithm proceeds by fully traversing the ring. When the message comes back to the node that started the election, the highest-ranked node from the live set is chosen as a leader. 

Consider processes 1,2,3,4,5,6. 6 - leader. 6 crashes. Let 3 start election.

3 -ping,{3}-> 4

4 -ping,{3,4}-> 5

5 -ping,{3,4,5}-> 6, 6 dead

5 -ping,{3,4,5}-> 1

1 -ping,{3,4,5,1}-> 2

2 -ping,{3,4,5,1,2}-> 3

3 -notify,{new leader 5}-> 4

...

etc 


Variants of this algorithm include collecting a single highest-ranked identifier instead of a set of active nodes to save space. When the algorithm comes back to the node that has started the election, the last known highest identifier is circulated across the ring once again.


Since the ring can be partitioned in two or more parts, with each part potentially electing its own leader, this approach **doesn’t hold a safety property**.